## Importing Libraries

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
from tqdm import tqdm
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## Setting up data

In [3]:
import json
with open("/content/drive/MyDrive/TUDelft/NLP Group Project/data/filtered_decomposed_claims.json") as f:
  train_data = json.load(f)
len(train_data)

2576

# BM25

In [4]:
pip install rank_bm25

In [5]:
import re
from rank_bm25 import BM25Okapi

In [6]:
with open("/content/drive/MyDrive/TUDelft/NLP Group Project/bm25/corpus_evidence_unified.json") as f:
  document_corpus = json.load(f)

In [7]:
def preprocess_text(text):
	text = text.lower()
	text = re.sub(r'\s\.\.\.', '', text)
	text = re.sub(r'\.', '', text)
	text = text.split(" ")
	return text

In [8]:
corpus = list(document_corpus.values())
tokenized_corpus = []
for doc in corpus:
	tokenized_corpus.append(preprocess_text(doc))

bm25 = BM25Okapi(tokenized_corpus)

## ABHISHEKS CODE

In [47]:
def run_bm25(query, top_k):
  results = bm25.get_top_n(preprocess_text(query), corpus, n=top_k)
  return results

In [48]:
def do_bm25(data, save_path, pick_up_from=None):

  features = []
  start_index = 0
  if pick_up_from:
      with open(pick_up_from, 'r') as file:
          features = json.load(file)
          start_index = start_index+len(features)  # Determine where to start processing
          print(f"Picking up from {start_index} claims")

  def save_progress(features):
        with open(save_path, 'w') as file:
            json.dump(features, file, indent=2)
        print("saved progress to ", save_path)

  # for i, example in enumerate(tqdm(data)):
  for i in tqdm(range(start_index, len(data)), desc="Processing Claims"):

    claim = data[i]['claim']
    q1 = data[i]['q1']
    q2 = data[i]['q2']
    q3 = data[i]['q3']
    results = []
    top_k = 3

    if q2 != "":
      top_k = 2
      results.extend(run_bm25(q2, top_k))

    results.extend(run_bm25(q1, top_k))
    results.extend(run_bm25(q3, top_k))

    evidences = ' '.join(results)
    feature_item = {
            "claim_evidence_string": f"[Claim]:{claim} [Evidences]:{evidences}",
            "idx": data[i]['idx'],
            "taxonomy_label": data[i]['taxonomy_label'],
            "label": data[i]['label']

        }
    features.append(feature_item)


    if (len(features) - start_index) % 10 == 0:
                save_progress(features)

  save_progress(features)


In [49]:
train_features = do_bm25(data=train_data,
                          save_path = '/content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_subclaims/bm25_features_train_set.json',
                          pick_up_from='/content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_subclaims/bm25_features_train_set.json'
                        )

Picking up from 146 claims


Processing Claims:   0%|          | 10/2430 [01:43<6:47:21, 10.10s/it]

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_subclaims/bm25_features_train_set.json


Processing Claims:   1%|          | 20/2430 [03:36<7:21:08, 10.98s/it]

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_subclaims/bm25_features_train_set.json


...

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_subclaims/bm25_features_train_set.json


Processing Claims:  23%|██▎       | 570/2430 [1:31:56<5:23:45, 10.44s/it]

saved progress to  /content/drive/MyDrive/TUDelft/NLP Group Project/bm25_of_subclaims/bm25_features_train_set.json


Processing Claims:  23%|██▎       | 571/2430 [1:32:10<5:00:07,  9.69s/it]


KeyboardInterrupt: 